<a href="https://colab.research.google.com/github/izzul112/Submission-Image_Classification/blob/master/Submission_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = '/content/gdrive/My Drive/dataset/dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp'
train_dir = os.path.join(base_dir, 'dataset')

In [4]:
os.listdir('/tmp/dataset')

['NORMAL', 'DRUSEN', 'CNV', 'DME']

In [5]:
# membuat direktori CNV pada direktori data training
train_CNV_dir = os.path.join(train_dir, 'CNV')

# membuat direktori DME pada direktori data training
train_DME_dir = os.path.join(train_dir, 'DME')

# membuat direktori DRUSEN pada direktori data training
train_DRUSEN_dir = os.path.join(train_dir, 'DRUSEN')

# membuat direktori NORMAL pada direktori data training
train_NORMAL_dir = os.path.join(train_dir, 'NORMAL')

# membuat direktori CNV pada direktori data validasi
validation_CNV_dir = os.path.join(train_dir, 'CNV')

# membuat direktori DME pada direktori data validasi
validation_DME_dir = os.path.join(train_dir, 'DME')

# membuat direktori DRUSEN pada direktori data validasi
validation_DRUSEN_dir = os.path.join(train_dir, 'DRUSEN')

# membuat direktori NORMAL pada direktori data validasi
validation_NORMAL_dir = os.path.join(train_dir, 'NORMAL')

In [6]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest',
                    validation_split=0.2)

In [7]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(200, 300),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena kita merupakan masalah klasifikasi 2 kelas lebih maka menggunakan class_mode = 'categorical'
        class_mode="categorical",
        subset='training'
        )

validation_generator = train_datagen.flow_from_directory(
        train_dir, # direktori data validasi
        target_size=(200, 300), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena kita merupakan masalah klasifikasi 2 kelas lebih maka menggunakan class_mode = 'categorical'
        class_mode='categorical',
        subset='validation'
        )

Found 9600 images belonging to 4 classes.
Found 2400 images belonging to 4 classes.


In [30]:
# Callback

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [31]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [32]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.RMSprop(),
              metrics=['accuracy'])

In [33]:
history = model.fit(
            train_generator,
            steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
            epochs=20, 
            validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
            validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
            verbose=2, callbacks=[callbacks])

Epoch 1/20
25/25 - 10s - loss: 1.4064 - accuracy: 0.2300 - val_loss: 1.3984 - val_accuracy: 0.1000
Epoch 2/20
25/25 - 10s - loss: 1.3921 - accuracy: 0.2600 - val_loss: 1.3863 - val_accuracy: 0.3000
Epoch 3/20
25/25 - 10s - loss: 1.3913 - accuracy: 0.2700 - val_loss: 1.3760 - val_accuracy: 0.2000
Epoch 4/20
25/25 - 10s - loss: 1.3888 - accuracy: 0.2600 - val_loss: 1.4096 - val_accuracy: 0.2500
Epoch 5/20
25/25 - 10s - loss: 1.3953 - accuracy: 0.2800 - val_loss: 1.3993 - val_accuracy: 0.1500
Epoch 6/20
25/25 - 9s - loss: 1.3921 - accuracy: 0.3200 - val_loss: 1.4054 - val_accuracy: 0.1000
Epoch 7/20
25/25 - 9s - loss: 1.3906 - accuracy: 0.2400 - val_loss: 1.3887 - val_accuracy: 0.2000
Epoch 8/20
25/25 - 9s - loss: 1.3896 - accuracy: 0.1800 - val_loss: 1.3863 - val_accuracy: 0.4000
Epoch 9/20
25/25 - 9s - loss: 1.3854 - accuracy: 0.3200 - val_loss: 1.3969 - val_accuracy: 0.1500
Epoch 10/20
25/25 - 9s - loss: 1.3846 - accuracy: 0.2800 - val_loss: 1.4100 - val_accuracy: 0.2000
Epoch 11/20
25

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()